In [4]:
import pandas as pd
import time
import ast

In [7]:
def convert_row_to_fpl_data(row):
    # Prep output list of lists
    output_data = []
    
    # Get universal data
    match_id = int(row["match_id"])
    season = str(row["season"])
    gameweek = str(row["gameweek"])
    data_start = [match_id,season,gameweek]
    
    # Column names for output format
    columns = ["match_id","season","gameweek","team","side","player_id","pos","mins","goals","assists","clean_sheet",
               "shot_saves","pen_saves","pen_misses","goals_conceded","yellow_cards","red_cards","own_goals"]
    
    # Get data needed to get relevant information
    subs = ast.literal_eval(row["subs"])
    goals = ast.literal_eval(row["goals"])
    cards = ast.literal_eval(row["cards"])
    hScore = int(row["hScore"])
    aScore = int(row["aScore"])
    pen_misses = ast.literal_eval(row["pen_misses"])
    
    # Get data for each of the home_players
    team = "H"
    side = str(row["hName"])
    home_team = ast.literal_eval(row["home_lineup"])
    home_bench = ast.literal_eval(row["home_bench"])
    home_players = [(int(x["id"]), str(x["pos"]), "start") for x in home_team] 
    home_players += [(int(y["id"]), str(y["pos"]), "bench") for y in home_bench]
    for player_data in home_players:
        player = player_data[0]
        pos = player_data[1]
        mins = calculate_minutes_played(player,subs,player_data[2])
        if mins > 0:
            goals_scored = calculate_goals_scored(player,goals)
            assists = calculate_assists(player,goals)
            clean_sheet = check_if_clean_sheet(player,mins,aScore)
            shot_saves = 0    #TODO - How to get this data?
            pen_saves = calculate_penalty_saves(player,subs,cards,pos,pen_misses)
            pens_missed = calculate_penalty_misses(player,pen_misses)
            goals_conceded = aScore
            yellow_cards, red_cards = count_yellow_and_red_cards(player,cards)
            own_goals = count_own_goals(player,goals)
            player_output_data = data_start + [team,side,player,pos,mins,goals_scored,assists,clean_sheet,shot_saves,
                                               pen_saves,pens_missed,goals_conceded,yellow_cards,red_cards,own_goals]
            output_data.append(player_output_data)
        
    # Get data for each of the away_players
    team = "A"
    side = str(row["aName"])
    away_team = ast.literal_eval(row["away_lineup"])
    away_bench = ast.literal_eval(row["away_bench"])
    away_players = [(int(x["id"]),str(x["pos"]),"start") for x in away_team] 
    away_players += [(int(y["id"]),str(y["pos"]),"bench") for y in away_bench]
    for player_data in away_players:
        player = player_data[0]
        pos = player_data[1]
        mins = calculate_minutes_played(player,subs,player_data[2])
        if mins > 0:
            goals_scored = calculate_goals_scored(player,goals)
            assists = calculate_assists(player,goals)
            clean_sheet = check_if_clean_sheet(player,mins,hScore)
            shot_saves = 0    #TODO - How to get this data?
            pen_saves = calculate_penalty_saves(player,subs,cards,pos,pen_misses)
            pens_missed = calculate_penalty_misses(player,pen_misses)
            goals_conceded = aScore
            yellow_cards, red_cards = count_yellow_and_red_cards(player,cards)
            own_goals = count_own_goals(player,goals)
            player_output_data = data_start + [team,side,player,pos,mins,goals_scored,assists,clean_sheet,shot_saves,
                                               pen_saves,pens_missed,goals_conceded,yellow_cards,red_cards,own_goals]
            output_data.append(player_output_data)
        
    output_df = pd.DataFrame(data=output_data,columns=columns)
    return output_df        

In [6]:
def calculate_minutes_played(player,subs,player_type):
    if player_type == "start":
        # So a starter
        for sub in subs:
            if int(player) == int(sub["player_id"]):
                minutes_off = int(str(sub["time"])[:2])
                return minutes_off
        return 90
    else:
        # So a bench player
        for sub in subs:
            if int(player) == int(sub["player_id"]):
                minutes_on = int(str(sub["time"])[:2])
                return int(90-minutes_on)
        return 0

def calculate_goals_scored(player,goals):
    goals_scored = 0
    if len(goals) == 0:
        return 0
    for goal in goals:
        if str(goal["type"]) != "Own Goal":
            if int(player) == int(goal["scorer_id"]):
                goals_scored += 1
    return goals_scored

def calculate_assists(player,goals):
    assists = 0
    if len(goals) == 0:
        return 0
    for goal in goals:
        if str(goal["type"]) != "Own Goal":
            try:
                if int(player) == int(goal["assist_id"]):
                    assists += 1
            except:
                assists = assists
    return assists

def calculate_penalty_saves(player,subs,cards,position,pen_misses):
    off_time = 90
    saves = 0
    if len(pen_misses) == 0:
        return 0
    if position == "G":
        # Figure out if goalkeeper subbed off
        for sub in subs:
            if int(sub["player_id"]) == int(player):
                off_time = int(str(sub["time"])[:2])
                break
        # If not, was goalkeeper sent off?
        if off_time == 90:
            for card in cards:
                if int(card["player_id"]) == int(player):
                    if str(card["player_id"])[:8] == "Red Card":
                        off_time = int(str(card["time"])[:2])
        # Use these details to decide if gk on pitch when pen was missed
        for pen in pen_misses:
            pen_time = int(str(pen["time"])[:2])
            if pen_time <= off_time:
                # Check if pen was saved or missed
                if str(pen["type"]) == "Pen Saved":
                    saves += 1
    return saves

def calculate_penalty_misses(player,pen_misses):
    misses = 0
    for pen in pen_misses:
        if int(player) == int(pen["player_id"]):
            misses += 1
    return misses

def count_yellow_and_red_cards(player,cards):
    yellow_cards = 0
    for card in cards:
        if int(player) == int(card["player_id"]):
            if str(card["type"]) == "Yellow Card":
                yellow_cards += 1
            elif str(card["type"])[:8] == "Red Card":
                return 0,1
    return yellow_cards,0

def count_own_goals(player,goals):
    own_goals = 0
    for goal in goals:
        if goal["type"] == "Own Goal":
            if int(goal["player_id"]) == int(player):
                own_goals += 1
    return own_goals

def check_if_clean_sheet(player,mins,score):
    if score == 0 and mins >= 60:
        return True
    return False